In [86]:
# !pip install pandas numpy spacecutter torch datamol skorch molfeat

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [88]:
# proj_dir =  'drive/MyDrive/Polaris_ASAP_competition/polaris_challenge/admet'
proj_dir = '/Users/robertarbon/Library/CloudStorage/GoogleDrive-robert.arbon@gmail.com/My Drive/Polaris_ASAP_competition/polaris_challenge/admet'


In [89]:
import pandas as pd
import numpy as np
from spacecutter.models import OrdinalLogisticModel
import torch
from torch import nn
import datamol as dm
import matplotlib.pyplot as plt

from skorch import NeuralNet
from skorch.dataset import Dataset
from skorch.helper import SkorchDoctor
from skorch.callbacks import EarlyStopping

from spacecutter.callbacks import AscensionCallback
from spacecutter.losses import CumulativeLinkLoss
from sklearn.metrics import mean_absolute_error
from scipy.stats import kendalltau

from sklearn.preprocessing import RobustScaler

# from molfeat.trans.pretrained.hf_transformers import PretrainedHFTransformer


In [90]:
# Imputed training data
df_imp = pd.read_csv(f'{proj_dir}/dm_features/ordinal_data_split_2/train_admet_split2_log_pmm_imputed.csv')
# Non-imputed validation data
df_val = pd.read_csv(f'{proj_dir}/dm_features/ordinal_data_split_2/train_admet_split2_features.csv')
# change names
df_val.rename(columns={'Molecule Name': 'Molecule.Name', 'LogMDR1-MDCKII':'LogMDR1.MDCKII'}, inplace=True)
df_imp.rename(columns={'Molecule Name': 'Molecule.Name', 'LogMDR1-MDCKII':'LogMDR1.MDCKII'}, inplace=True)

# Smiles columns because they were removed (for some unknown reason)
df_smiles = pd.read_csv(f'{proj_dir}/data/train_admet_all.csv')
df_smiles.rename(columns={'Molecule Name': 'Molecule.Name', 'LogMDR1-MDCKII':'LogMDR1.MDCKII'}, inplace=True)


df_imp = df_imp.merge(df_smiles.loc[:, ['Molecule.Name', 'CXSMILES']], on='Molecule.Name', how='left')
df_val = df_val.merge(df_smiles.loc[:, ['Molecule.Name', 'CXSMILES']], on='Molecule.Name', how='left')

In [93]:
scaler = RobustScaler()
hasattr(scaler, 'n_features_in')
# scaler.transform(X=np.random.random((1/0, 10)))

False

In [94]:
from math import isnan

def digitize_y(y_s, n_cuts=None, bins_by_target=None):
    results = {}
    targets = list(y_s.columns)
    for target in targets:
        y = y_s.loc[:, target].values
        missing_ix = ~np.isnan(y)
        y = y[missing_ix]
        max_y = np.max(y)
        n_y = y.shape[0]
        if bins_by_target is None:
          if n_cuts is None:
              bins =list(np.unique(np.sort(y)))
          else:
              step = int(np.unique(y).shape[0]//n_cuts)
              bins = list(np.unique(np.sort(y))[::step])
        else:
          bins = list(bins_by_target[target]['bins'])

        if bins[-1] != max_y:
          bins.append(max_y)
          bins.sort()

        y_ord, bins = pd.cut(y, bins=bins, include_lowest=True, labels=False, retbins=True, ordered=True, )
        assert y.shape[0] == y_ord.shape[0], f"{target} {y.shape[0]} {y_ord.shape[0]}"
        # assert np.isnan(y).sum() == 0
        if np.isnan(y_ord).sum() > 0:
          print(f"{target} {y.min()}, {y.max()} {bins[0]} {bins[-1]}")


        results[target] = {'values': y_ord, 'bins': bins, 'missing_ix': missing_ix }

    return results

def get_Xy(df, ix, n_cuts, bins_by_target=None, features=None, scalers_by_feature=None):
    if isinstance(features, str):
      features = [features]
    X_all = []

    if scalers_by_feature is None: 
       scalers_by_feature = {}

    for feature in features:
      if feature == 'fp':
        print('using morgan fingerprints')
        smiles = df.loc[ix, 'CXSMILES'].values
        X = np.vstack([dm.to_fp(dm.to_mol(smi)) for smi in smiles])
      elif feature == 'rdkit_simple':
        print('using rdkit simple')
        Xraw = df.loc[ix, ['clogp', 'fsp3', 'mw', 'n_aliphatic_carbocycles',
          'n_aliphatic_heterocyles', 'n_aliphatic_rings',
          'n_aromatic_carbocycles', 'n_aromatic_heterocyles', 'n_aromatic_rings',
          'n_heavy_atoms', 'n_hetero_atoms', 'n_lipinski_hba', 'n_lipinski_hbd',
          'n_rings', 'n_rotatable_bonds', 'n_saturated_carbocycles',
          'n_saturated_heterocyles', 'n_saturated_rings', 'qed', 'sas', 'tpsa']].values
        
        if feature in scalers_by_feature:
          scaler = scalers_by_feature[feature] 
          X  = scaler.transform(Xraw)
        else:
          scaler = RobustScaler()
          X = scaler.fit_transform(Xraw)
          scalers_by_feature[feature] = scaler

      elif feature == 'chemp_prop':
        print('using chemprop features')
        chem_prop_features = pd.read_csv(f'{proj_dir}/chem_prop_features/train_admet_chemprop.csv')
        Xdf = df.loc[ix, :].merge(chem_prop_features, on='Molecule.Name', how='left')
        Xraw = Xdf.loc[:, chem_prop_features.columns.difference(['Molecule.Name', 'split'])]
        if feature in scalers_by_feature:
          scaler = scalers_by_feature[feature] 
          X  = scaler.transform(Xraw)
        else:
          scaler = RobustScaler()
          X = scaler.fit_transform(Xraw)
          scalers_by_feature[feature] = scaler
      elif feature == 'chemberta':
        print('using chemberta')
        chemberta = pd.read_csv(f'{proj_dir}/deep_ord/chemberta_77M_mtr.csv')
        Xdf = df.loc[ix, :].merge(chemberta, on='Molecule.Name', how='left')
        Xraw= Xdf.loc[:, chemberta.columns.difference(['Molecule.Name'])]
        if feature in scalers_by_feature:
          scaler = scalers_by_feature[feature] 
          X  = scaler.transform(Xraw)
        else:
          scaler = RobustScaler()
          X = scaler.fit_transform(Xraw)
          scalers_by_feature[feature] = scaler      
      else:
        raise ValueError(f"Unknown features: {features}")
      X_all.append(X)
    X = np.hstack(X_all)

    ys = df.filter(regex='^Log').loc[ix, :]
    assert ys.shape[0] == X.shape[0]
    y_dig = digitize_y(ys, n_cuts=n_cuts, bins_by_target=bins_by_target)
    return X, y_dig, scalers_by_feature

def train_data(df_train, imp_ix, df_val, n_cuts=None, features='fp'):
    ix_by_imp = None
    if imp_ix is None:
      train_ix = (df_train['split'] == 'train' ) & (df_train['.imp'] != 0)
      max_imp = df_train['.imp'].max()
      ix_by_imp = {i: df_train.loc[train_ix, '.imp'] == i for i in range(1, max_imp+1)}
    else:
      train_ix = (df_train['.imp'] == imp_ix) & (df_train['split'] == 'train')

    val_ix = df_val['split'] == 'val'

    results = []
    X_train, y_train_by_targ, scalers_by_feature = get_Xy(df_train, train_ix, n_cuts, features=features)
    X_val, y_val_by_targ, _ = get_Xy(df_val, val_ix, n_cuts, y_train_by_targ, features=features, scalers_by_feature=scalers_by_feature)

    results.append((X_train, y_train_by_targ, ix_by_imp))
    results.append((X_val, y_val_by_targ))

    return results

In [95]:
train, val = train_data(df_train=df_imp,
                        imp_ix=1,
                        df_val=df_val,
                        n_cuts=None, features=['rdkit_simple', 'chemp_prop'])

targets = list(train[1].keys())

training_by_target = {}

for target in targets:
  num_ds = 1
  ix_by_imp = train[2]
  if ix_by_imp is not None:
    num_ds = len(ix_by_imp)

  # accumulators
  all_y_hat = []
  all_epochs = []

  # All imputed training datasets
  all_X = train[0]
  all_y = train[1][target]['values'].reshape(-1, 1)

  # only keep features which are different
  keep_ix = np.std(all_X, axis=0)>0
  all_X = all_X[:, keep_ix].astype(np.float32)

  # Get validation data (not imputed so only done once. contains missing values)
  missing_ix = val[1][target]['missing_ix']
  Xval = val[0]

  Xval = Xval[missing_ix, :].astype(np.float32)
  Xval = Xval[:, keep_ix]
  # validation y values have been digitized using all the training y values.
  # so should be consistent.
  yval = val[1][target]['values'].reshape(-1, 1)
  bins = train[1][target]['bins']

  print(f'{target}')
  for i in range(num_ds):
    if i % 10 == 0:
      print(f'\t{i}/{num_ds}', end=',')

    # Get imputed training dataset.
    imp_ix = ix_by_imp[i+1] if ix_by_imp is not None else np.arange(all_X.shape[0]) # The zeroth imputed dataset is the original data.
    X = all_X[imp_ix, :]
    y = train[1][target]['values'].reshape(-1, 1)
    y = y[imp_ix]

    # print(f"{target}:\n\tn_train_obs: {X.shape[0]}, n_val_obs: {Xval.shape[0]} n_preds: {X.shape[1]}")

    # Stack all data for convenience.
    train_v_X = np.vstack([X, Xval])
    train_v_y = np.vstack([y, yval])
    train_ix = np.arange(X.shape[0])
    val_ix = np.arange(X.shape[0], train_v_X.shape[0])

    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    predictor = nn.Sequential(
        nn.Linear(num_features, num_features),
        nn.ReLU(),
        nn.Linear(num_features, num_features),
        nn.ReLU(),
        nn.Linear(num_features, 1),
    )


    skorch_model = NeuralNet(
        module=OrdinalLogisticModel,
        module__predictor=predictor,
        module__num_classes=num_classes,
        criterion=CumulativeLinkLoss,
        optimizer=torch.optim.Adam,
        train_split=lambda ds, y: (torch.utils.data.Subset(ds, train_ix),
                                  torch.utils.data.Subset(ds, val_ix)),
        callbacks=[
            ('ascension', AscensionCallback()),
            ('early_stopping', EarlyStopping(threshold=0.0001, load_best=True,
                                            patience=100))
        ],
        verbose=0,
        batch_size=X.shape[0],
        max_epochs=500,

    )

    skorch_model.fit(train_v_X, train_v_y)


    y_hat = np.argmax(skorch_model.predict(Xval), axis=1)


    all_y_hat.append(bins[y_hat])

    for i in range(len(skorch_model.history)-1, -1, -1):
      batch = skorch_model.history[i]

      if batch['valid_loss_best']:
        all_epochs.append(batch['epoch'])
        break

  # mean prediction over imputed datasets.
  mean_y_hat = np.mean(np.vstack(all_y_hat), axis=0)
  y_val_cont = bins[yval.reshape(-1)]

  print(f"\tmae: {mean_absolute_error(y_val_cont, mean_y_hat):4.2f}")
  print(f"\tmean epoches: {np.mean(all_epochs)}")

using rdkit simple
using chemprop features
using rdkit simple
using chemprop features
LogD
	0/1,Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kern

In [ ]:
# train, val = train_data(df_train=df_imp,
#                         imp_ix=None,
#                         df_val=df_val,
#                         n_cuts=None, features=['rdkit_simple', 'chemp_prop'])

# targets = list(train[1].keys())

# training_by_target = {}
# n_imp_ds = df_imp['.imp'].max()

# for target in targets:
#   all_X = train[0]
#   all_y = train[1][target]['values'].reshape(-1, 1)

#   # only keep features which are different
#   keep_ix = np.std(all_X, axis=0)>0
#   all_X = all_X[:, keep_ix].astype(np.float32)

#   # Get imputed training dataset.
#   shuffle_ix = np.random.choice(np.arange(all_X.shape[0]), size=all_X.shape[0], replace=False)
#   X = all_X[shuffle_ix, :]
#   y = all_y[shuffle_ix]

#   # Get validation data (not imputed so only done once. contains missing values)
#   missing_ix = val[1][target]['missing_ix']
#   Xval = val[0]
#   Xval = Xval[missing_ix, :].astype(np.float32)
#   Xval = Xval[:, keep_ix]
#   # validation y values have been digitized using all the training y values.
#   # so should be consistent.
#   yval = val[1][target]['values'].reshape(-1, 1)
#   bins = train[1][target]['bins']

#   # batch size
#   batch_size = all_X.shape[0]//n_imp_ds

#   # Stack all data for convenience.
#   train_v_X = np.vstack([X, Xval])
#   train_v_y = np.vstack([y, yval])
#   train_ix = np.arange(X.shape[0])
#   val_ix = np.arange(X.shape[0], train_v_X.shape[0])

#   # Model dimensions
#   num_features = X.shape[1]
#   num_classes = len(np.unique(y))

#   # Simple predictor
#   predictor = nn.Sequential(
#     nn.Linear(num_features, num_features),
#     nn.ReLU(),
#     nn.Linear(num_features, num_features),
#     nn.ReLU(),
#     nn.Linear(num_features, 1),
#   )

#   # Model
#   skorch_model = NeuralNet(
#     module=OrdinalLogisticModel,
#     module__predictor=predictor,
#     module__num_classes=num_classes,
#     criterion=CumulativeLinkLoss,
#     optimizer=torch.optim.Adam,
#     train_split=lambda ds, y: (torch.utils.data.Subset(ds, train_ix),
#                               torch.utils.data.Subset(ds, val_ix)),
#     callbacks=[
#         ('ascension', AscensionCallback()),
#         ('early_stopping', EarlyStopping(threshold=0.0001, load_best=True,
#                                         patience=100))
#     ],
#     verbose=0,
#     batch_size=X.shape[0],
#     max_epochs=500,

#   )
#   # Fit
#   skorch_model.fit(train_v_X, train_v_y)

#   # predict on validation
#   y_hat_ord = np.argmax(skorch_model.predict(Xval), axis=1)
#   y_hat = bins[y_hat_ord]
#   y_val = bins[yval.reshape(-1)]

#   print(f"\tmae: {mean_absolute_error(y_val, y_hat):4.2f}")

#   # Find best epoch
#   for i in range(len(skorch_model.history)-1, -1, -1):
#     batch = skorch_model.history[i]

#     if batch['valid_loss_best']:
#       print(f"\tepochs: {batch['epoch']}/{len(skorch_model.history)}")
#       break



using rdkit simple
using chemprop features
using rdkit simple
using chemprop features
	mae: 0.61
	epochs: 12/112
	mae: 0.45
	epochs: 6/106
	mae: 0.51
	epochs: 33/133
	mae: 0.59
	epochs: 20/120
	mae: 0.87
	epochs: 23/123
